<a href="https://colab.research.google.com/github/GabrielPSMartins/rna_com_keras/blob/main/lab_rna_com_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.callbacks import EarlyStopping

In [89]:
df = pd.read_csv("data/Churn_treino.csv", sep=";")
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,10134888,1
1,608,Spain,Female,41,1,8380786,1,0,1,11254258,0
2,502,France,Female,42,8,1596608,3,1,0,11393157,1
3,699,France,Female,39,1,0,2,0,0,9382663,0
4,850,Spain,Female,43,2,12551082,1,1,1,790841,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0,2,1,0,9627064,0
9996,516,France,Male,35,10,5736961,1,1,1,10169977,0
9997,709,France,Female,36,7,0,1,0,1,4208558,1
9998,772,Germany,Male,42,3,7507531,2,1,0,9288852,1


In [90]:
X = df.drop("Exited", axis=1)
y = df["Exited"]

In [91]:
standardscaler = StandardScaler()
numerical = X.select_dtypes(include=['float64', 'int64']).columns
X[numerical] = standardscaler.fit_transform(X[numerical])

In [92]:
labelencoder = LabelEncoder()
categorical = X.select_dtypes(include='object').columns
for col in categorical:
  X[col] = labelencoder.fit_transform(X[col])

In [94]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=0)

In [95]:
modelo = Sequential()
modelo.add(Input(shape=(X_treino.shape[1],)))

modelo.add(Dense(units=64, activation='relu'))
modelo.add(Dropout(0.3))

modelo.add(Dense(units=32, activation='relu'))
modelo.add(Dropout(0.3))

modelo.add(Dense(units=16, activation='relu'))
modelo.add(Dropout(0.3))

modelo.add(Dense(units=1, activation='sigmoid'))

In [ ]:
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

modelo.fit(X_treino, y_treino, epochs=100, batch_size=32, validation_split=0.2, callbacks=[callback])

In [97]:
previsoes = modelo.predict(X_teste)
previsoes

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[0.19180089],
       [0.23864849],
       [0.24018139],
       ...,
       [0.07392905],
       [0.09126621],
       [0.4644624 ]], dtype=float32)

In [82]:
y_pred = (previsoes > 0.5).astype('int32')
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]], dtype=int32)

In [98]:
print('Acurácia:', accuracy_score(y_teste, y_pred))
print('F1:', f1_score(y_teste, y_pred))
print('Recall:', recall_score(y_teste, y_pred))
print('Matriz de Confusão:\n', confusion_matrix(y_teste, y_pred))

Acurácia: 0.8576666666666667
F1: 0.5882352941176471
Recall: 0.49114331723027377
Matriz de Confusão:
 [[2268  111]
 [ 316  305]]
